In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U -q kaggle

In [3]:
!mkdir -p ~/.kaggle

In [4]:
from google.colab import files

In [5]:
files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"miguelariaslondoo","key":"7418f5016cd15765af60765f150c1ba7"}'}

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle competitions download -c champs-scalar-coupling

 97% 367M/377M [00:02<00:00, 172MB/s]
100% 377M/377M [00:02<00:00, 141MB/s]


In [ ]:
!unzip -o '/content/champs-scalar-coupling.zip'

In [9]:
import glob
import pandas as pd

csv_files = glob.glob('*.csv')

for file in csv_files:
    df = pd.read_csv(file)
    print(f"Information for {file}:")
    print(f"Number of rows: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Data types:\n{df.dtypes}\n")


Information for potential_energy.csv:
Number of rows: 130789
Columns: ['molecule_name', 'potential_energy']
Data types:
molecule_name        object
potential_energy    float64
dtype: object

Information for magnetic_shielding_tensors.csv:
Number of rows: 2358875
Columns: ['molecule_name', 'atom_index', 'XX', 'YX', 'ZX', 'XY', 'YY', 'ZY', 'XZ', 'YZ', 'ZZ']
Data types:
molecule_name     object
atom_index         int64
XX               float64
YX               float64
ZX               float64
XY               float64
YY               float64
ZY               float64
XZ               float64
YZ               float64
ZZ               float64
dtype: object

Information for structures.csv:
Number of rows: 2358875
Columns: ['molecule_name', 'atom_index', 'atom', 'x', 'y', 'z']
Data types:
molecule_name     object
atom_index         int64
atom              object
x                float64
y                float64
z                float64
dtype: object

Information for scalar_coupling_contributio

In [10]:
for file in csv_files:
    df = pd.read_csv(file)
    print(f"{file}:")

potential_energy.csv:
magnetic_shielding_tensors.csv:
structures.csv:
scalar_coupling_contributions.csv:
test.csv:
dipole_moments.csv:
train.csv:
sample_submission.csv:
mulliken_charges.csv:


## ORGANIZAMOS LOS DATOS ADICIONALES PARA AGREGARLOS AL DATAFRAME DE TRAIN Y TEST

In [11]:

# Load the dipole_moments.csv file into a pandas dataframe
df_dipole = pd.read_csv('dipole_moments.csv')
# Rename the columns
df_dipole = df_dipole.rename(columns={'molecule_name': 'molecule_name', 'X': 'dipole_X','Y': 'dipole_Y', 'Z': 'dipole_Z'})
# Load the potential_energy.csv file into a pandas dataframe
df_potential = pd.read_csv('potential_energy.csv')
# Merge the two dataframes on the 'molecule_name' column
df_dipole_energy = pd.merge(df_dipole, df_potential, on='molecule_name')

# Print the merged dataframe
print(df_dipole_energy.head())
print(df_dipole_energy.shape)

      molecule_name  dipole_X  dipole_Y  dipole_Z  potential_energy
0  dsgdb9nsd_000001    0.0000       0.0    0.0000        -40.523680
1  dsgdb9nsd_000002   -0.0002       0.0    1.6256        -56.560246
2  dsgdb9nsd_000003    0.0000       0.0   -1.8511        -76.426077
3  dsgdb9nsd_000004    0.0000       0.0    0.0000        -77.335268
4  dsgdb9nsd_000005    0.0000       0.0   -2.8937        -93.428488
(130789, 5)


In [19]:
df_magshield = pd.read_csv('magnetic_shielding_tensors.csv')
df_mulliken = pd.read_csv('mulliken_charges.csv')
df_structures = pd.read_csv('structures.csv')

print(f"shape of magshield {df_magshield.shape}")
print(f"shape of mulliken {df_mulliken.shape}")
print(f"shape of structures {df_structures.shape}")

df_merged = pd.merge(df_magshield, df_mulliken)
df_merged = pd.merge(df_merged, df_structures)
df_merged = df_merged.rename(columns={'x': 'struc_x', 'y': 'struc_y', 'z': 'struc_z'})
print(df_merged.head())
print(df_merged.shape)

shape of magshield (2358875, 11)
shape of mulliken (2358875, 3)
shape of structures (2358875, 6)
      molecule_name  atom_index        XX      YX      ZX      XY        YY  \
0  dsgdb9nsd_000001           0  195.3147  0.0000 -0.0001  0.0000  195.3171   
1  dsgdb9nsd_000001           1   31.3410 -1.2317  4.0544 -1.2317   28.9546   
2  dsgdb9nsd_000001           2   31.5814  1.2173 -4.1474  1.2173   28.9036   
3  dsgdb9nsd_000001           3   31.5172  4.1086  1.2723  4.1088   33.9068   
4  dsgdb9nsd_000001           4   31.4029 -4.0942 -1.1793 -4.0944   34.0776   

       ZY      XZ      YZ        ZZ  mulliken_charge atom   struc_x   struc_y  \
0  0.0007 -0.0001  0.0007  195.3169        -0.535689    C -0.012698  1.085804   
1 -1.7173  4.0546 -1.7173   34.0861         0.133921    H  0.002150 -0.006031   
2 -1.6036 -4.1476 -1.6036   33.8967         0.133922    H  1.011731  1.463751   
3  1.6950  1.2724  1.6951   28.9579         0.133923    H -0.540815  1.447527   
4  1.6259 -1.1795  1.62

In [20]:
# Load the train.csv file into a pandas dataframe
df_train = pd.read_csv('train.csv')
df_scalar = pd.read_csv('scalar_coupling_contributions.csv')

# Merge the two dataframes based on molecule name and atom_index_0
df_train_merged = pd.merge(df_train, df_scalar)
df_train_merged = df_train_merged.drop('id', axis=1)

print(df_train_merged.head())
print(df_train_merged.shape)

      molecule_name  atom_index_0  atom_index_1  type  \
0  dsgdb9nsd_000001             1             0  1JHC   
1  dsgdb9nsd_000001             1             2  2JHH   
2  dsgdb9nsd_000001             1             3  2JHH   
3  dsgdb9nsd_000001             1             4  2JHH   
4  dsgdb9nsd_000001             2             0  1JHC   

   scalar_coupling_constant       fc        sd      pso       dso  
0                   84.8076  83.0224  0.254579  1.25862  0.272010  
1                  -11.2570 -11.0347  0.352978  2.85839 -3.433600  
2                  -11.2548 -11.0325  0.352944  2.85852 -3.433870  
3                  -11.2543 -11.0319  0.352934  2.85855 -3.433930  
4                   84.8074  83.0222  0.254585  1.25861  0.272013  
(4659076, 9)


In [27]:
# Distribution of the categorical column "type"
import pandas as pd

# Load the train.csv file into a pandas dataframe
dftrain = pd.read_csv('train.csv')

# Get all the unique values in the 'type' column
types = dftrain['type'].unique()

# Print the unique values
print(types)
# Get the value counts of each unique value in the 'type' column
type_counts = dftrain['type'].value_counts()

# Calculate the percentage of each unique value
type_percentages = type_counts / len(dftrain) * 100

# Print the percentage of each unique value
print(type_percentages)


['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']
3JHC    32.435766
2JHC    24.486980
1JHC    15.220464
3JHH    12.674809
2JHH     8.112939
3JHN     3.576095
2JHN     2.555421
1JHN     0.937525
Name: type, dtype: float64


In [28]:
#Missing values in the df
import pandas as pd
import matplotlib.pyplot as plt

# Load the train.csv file into a pandas dataframe
dftrain = pd.read_csv('train.csv')
dftest = pd.read_csv('test.csv')

# Check the shape of the dataframes
print(f"shape of train {dftrain.shape}")
print(f"shape of test {dftest.shape}")
# Check the data types of each column

# Check for missing values
print(dftest.isnull().sum())
print(dftrain.isnull().sum())


shape of train (4659076, 6)
shape of test (2505190, 5)
id               0
molecule_name    0
atom_index_0     0
atom_index_1     0
type             0
dtype: int64
id                          0
molecule_name               0
atom_index_0                0
atom_index_1                0
type                        0
scalar_coupling_constant    0
dtype: int64


In [21]:


# Create copies of df_merged for atom index 0 and atom index 1
df_merged_atomindex0 = df_merged.copy()
df_merged_atomindex1 = df_merged.copy()

# Add suffixes to the columns of the copies to differentiate between atom index 0 and atom index 1
df_merged_atomindex0 = df_merged_atomindex0.add_suffix('_atomIndex0')
df_merged_atomindex1 = df_merged_atomindex1.add_suffix('_atomIndex1')

# Rename specific columns for clarity
df_merged_atomindex0 = df_merged_atomindex0.rename(columns={'molecule_name_atomIndex0': 'molecule_name', 'atom_index_atomIndex0': 'atom_index'})
df_merged_atomindex1 = df_merged_atomindex1.rename(columns={'molecule_name_atomIndex1': 'molecule_name', 'atom_index_atomIndex1': 'atom_index'})

In [22]:


# Merge df_train_merged with df_merged_atomindex0 based on molecule_name and atom_index_0
df_train_merged_final = pd.merge(df_train_merged, df_merged_atomindex0, how='left', left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name', 'atom_index'])

# Drop the redundant 'atom_index' column after the merge
df_train_merged_final = df_train_merged_final.drop(['atom_index'], axis=1)

# Merge df_train_merged_final with df_merged_atomindex1 based on molecule_name and atom_index_1
df_train_merged_final = pd.merge(df_train_merged_final, df_merged_atomindex1, how='left', left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name', 'atom_index'])

# Drop the redundant 'atom_index' column after the second merge
df_train_merged_final = df_train_merged_final.drop(['atom_index'], axis=1)

# Print the first few rows of the merged dataframe
print(df_train_merged_final.head())

# Print the shape of the merged dataframe
print(df_train_merged_final.shape)


      molecule_name  atom_index_0  atom_index_1  type  \
0  dsgdb9nsd_000001             1             0  1JHC   
1  dsgdb9nsd_000001             1             2  2JHH   
2  dsgdb9nsd_000001             1             3  2JHH   
3  dsgdb9nsd_000001             1             4  2JHH   
4  dsgdb9nsd_000001             2             0  1JHC   

   scalar_coupling_constant       fc        sd      pso       dso  \
0                   84.8076  83.0224  0.254579  1.25862  0.272010   
1                  -11.2570 -11.0347  0.352978  2.85839 -3.433600   
2                  -11.2548 -11.0325  0.352944  2.85852 -3.433870   
3                  -11.2543 -11.0319  0.352934  2.85855 -3.433930   
4                   84.8074  83.0222  0.254585  1.25861  0.272013   

   XX_atomIndex0  ...  YY_atomIndex1  ZY_atomIndex1  XZ_atomIndex1  \
0        31.3410  ...       195.3171         0.0007        -0.0001   
1        31.3410  ...        28.9036        -1.6036        -4.1476   
2        31.3410  ...        33.

In [26]:
print(f"Columns: {df_train_merged_final.columns.tolist()}")
print({len(df_train_merged_final.columns.tolist())})
print(df_train_merged_final.shape)

Columns: ['molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant', 'fc', 'sd', 'pso', 'dso', 'XX_atomIndex0', 'YX_atomIndex0', 'ZX_atomIndex0', 'XY_atomIndex0', 'YY_atomIndex0', 'ZY_atomIndex0', 'XZ_atomIndex0', 'YZ_atomIndex0', 'ZZ_atomIndex0', 'mulliken_charge_atomIndex0', 'atom_atomIndex0', 'struc_x_atomIndex0', 'struc_y_atomIndex0', 'struc_z_atomIndex0', 'XX_atomIndex1', 'YX_atomIndex1', 'ZX_atomIndex1', 'XY_atomIndex1', 'YY_atomIndex1', 'ZY_atomIndex1', 'XZ_atomIndex1', 'YZ_atomIndex1', 'ZZ_atomIndex1', 'mulliken_charge_atomIndex1', 'atom_atomIndex1', 'struc_x_atomIndex1', 'struc_y_atomIndex1', 'struc_z_atomIndex1']
{37}
(4659076, 37)


In [17]:
df_train_merged_final.to_csv('df_train_merged_final.csv', index=False)

In [25]:
print("df_train_merged_final.csv:")
print(f"Number of rows: {len(df_train_merged_final)}")
print(f"Number of Columns: {len((df_train_merged_final.columns.tolist()))}")
print(f"Data types:\n{df_train_merged_final.dtypes}\n")


df_train_merged_final.csv:
Number of rows: 4659076
Number of Columns: 37
Data types:
molecule_name                  object
atom_index_0                    int64
atom_index_1                    int64
type                           object
scalar_coupling_constant      float64
fc                            float64
sd                            float64
pso                           float64
dso                           float64
XX_atomIndex0                 float64
YX_atomIndex0                 float64
ZX_atomIndex0                 float64
XY_atomIndex0                 float64
YY_atomIndex0                 float64
ZY_atomIndex0                 float64
XZ_atomIndex0                 float64
YZ_atomIndex0                 float64
ZZ_atomIndex0                 float64
mulliken_charge_atomIndex0    float64
atom_atomIndex0                object
struc_x_atomIndex0            float64
struc_y_atomIndex0            float64
struc_z_atomIndex0            float64
XX_atomIndex1                 float64
YX_

In [24]:
import numpy as np
# Select 6 random columns
cols = df_train_merged_final.sample(n=6, axis=1).columns

# Replace 5% of the data in those columns with NaN
for col in cols:
    df_train_merged_final.loc[df_train_merged_final.sample(frac=0.05).index, col] = np.nan
